https://python.langchain.com/v0.2/docs/how_to/graph_constructing/

In [3]:
# Common data processing
import textwrap
import pandas as pd

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_community.chat_models import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.document_loaders import TextLoader
from langchain.llms.base import LLM
from typing import List
import ollama
# Warning control
import warnings
warnings.filterwarnings("ignore")
from langchain_core.documents import Document
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

In [4]:
class OllamaLLM(LLM):
    """Custom LLM class to interface with the local Ollama model."""

    # model_name: str = "llama3.2"

    def _call(self, prompt: str, stop: List[str] = None) -> str:
        """Generate a response from the Ollama model."""
        result = ollama.chat(model=self.model_name, messages=[{'role': 'user', 'content': prompt}], stream=False)
        
        if isinstance(result, dict) and 'message' in result and 'content' in result['message']:
            return result['message']['content']
        else:
            raise ValueError(f"Failed to retrieve response from Ollama. Result: {result}")

    @property
    def _identifying_params(self) -> dict:
        """Return the identifying parameters of the model."""
        return {"model_name": self.model_name}
    
    @property
    def _llm_type(self) -> str:
        """Return the type of the LLM as required by the LangChain framework."""
        return "ollama"
    
llm = OllamaLLM(model_name='llama3.1')

# Connect to Neo4J Database

In [5]:
# Connect to local Neo4J database

NEO4J_URI = "neo4j://localhost"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "admin123"
NEO4J_DATABASE = "neo4j"

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

# Dataset Preparation

In [6]:
loader = PDFPlumberLoader("data/ISBANK2023.pdf")
docs = loader.load()
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)
# Check the number of pages
print("Number of pages in the PDF:",len(docs))
print("Number of documents after chunking:",len(documents))

# Uncomment to create a single document by combining all documents
# full_text = ""
# for row in docs:
#     if len(row.page_content) >= 500:
#         full_text = full_text + row.page_content


Number of pages in the PDF: 237
Number of documents after chunking: 710


In [42]:
for doc in documents:
    if "\n" in doc.page_content:
        print(doc)
        doc.page_content = doc.page_content.replace("\n", "")

for i, doc in enumerate(documents):
    if len(doc.page_content) <= 10:
        documents.pop(i)

In [4]:
full_text = full_text[50000:70000]

In [7]:
# import json
# import uuid
# from operator import itemgetter
# from typing import (
#     Any,
#     Callable,
#     Dict,
#     List,
#     Optional,
#     Sequence,
#     Type,
#     TypedDict,
#     TypeVar,
#     Union,
# )

# from langchain_community.chat_models.ollama import ChatOllama
# from langchain_core._api import deprecated
# from langchain_core.callbacks import (
#     AsyncCallbackManagerForLLMRun,
#     CallbackManagerForLLMRun,
# )
# from langchain_core.language_models import LanguageModelInput
# from langchain_core.messages import (
#     AIMessage,
#     BaseMessage,
#     ToolCall,
# )
# from langchain_core.output_parsers.base import OutputParserLike
# from langchain_core.output_parsers.json import JsonOutputParser
# from langchain_core.output_parsers.pydantic import PydanticOutputParser
# from langchain_core.outputs import ChatGeneration, ChatResult
# from langchain_core.prompts import SystemMessagePromptTemplate
# from langchain_core.runnables import Runnable, RunnableLambda
# from langchain_core.runnables.base import RunnableMap
# from langchain_core.runnables.passthrough import RunnablePassthrough
# from langchain_core.tools import BaseTool
# from langchain_core.utils.pydantic import is_basemodel_subclass
# from pydantic import (
#     BaseModel,
# )

# DEFAULT_SYSTEM_TEMPLATE = """You have access to the following tools:
# """  # noqa: E501

# DEFAULT_RESPONSE_FUNCTION = {
# }

# _BM = TypeVar("_BM", bound=BaseModel)
# _DictOrPydantic = Union[Dict, _BM]


# def _is_pydantic_class(obj: Any) -> bool:
#     return isinstance(obj, type) and (
#         is_basemodel_subclass(obj) or BaseModel in obj.__bases__
#     )


# def convert_to_ollama_tool(tool: Any) -> Dict:
#     return definition


# class _AllReturnType(TypedDict):
#     raw: BaseMessage
#     parsed: Optional[_DictOrPydantic]
#     parsing_error: Optional[BaseException]


# def parse_response(message: BaseMessage) -> str:
#     raise ValueError(f"`message` is not an instance of `AIMessage`: {message}")


# @deprecated(  # type: ignore[arg-type]
#     since="0.0.64", removal="1.0", alternative_import="langchain_ollama.ChatOllama"
# )
# class OllamaFunctions(ChatOllama):
#     """Function chat model that uses Ollama API."""

#     tool_system_prompt_template: str = DEFAULT_SYSTEM_TEMPLATE

#     def __init__(self, **kwargs: Any) -> None:
#         super().__init__(**kwargs)

#     def bind_tools(
#         self,
#         tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
#         **kwargs: Any,
#     ) -> Runnable[LanguageModelInput, BaseMessage]:
#         return self.bind(functions=tools, **kwargs)

#     def with_structured_output(
#         self,
#         schema: Union[Dict, Type[BaseModel]],
#         *,
#         include_raw: bool = False,
#         **kwargs: Any,
#     ) -> Runnable[LanguageModelInput, Union[Dict, BaseModel]]:
#         """Model wrapper that returns outputs formatted to match the given schema.

#         Args:
#             schema: The output schema as a dict or a Pydantic class. If a Pydantic class
#                 then the model output will be an object of that class. If a dict then
#                 the model output will be a dict. With a Pydantic class the returned
#                 attributes will be validated, whereas with a dict they will not be.
#             include_raw: If False then only the parsed structured output is returned. If
#                 an error occurs during model output parsing it will be raised. If True
#                 then both the raw model response (a BaseMessage) and the parsed model
#                 response will be returned. If an error occurs during output parsing it
#                 will be caught and returned as well. The final output is always a dict
#                 with keys "raw", "parsed", and "parsing_error".

#         Returns:
#             A Runnable that takes any ChatModel input and returns as output:

#                 If include_raw is True then a dict with keys:
#                     raw: BaseMessage
#                     parsed: Optional[_DictOrPydantic]
#                     parsing_error: Optional[BaseException]

#                 If include_raw is False then just _DictOrPydantic is returned,
#                 where _DictOrPydantic depends on the schema:

#                 If schema is a Pydantic class then _DictOrPydantic is the Pydantic
#                     class.

#                 If schema is a dict then _DictOrPydantic is a dict.

#         Example: Pydantic schema (include_raw=False):
#             .. code-block:: python

#                 from langchain_experimental.llms import OllamaFunctions
#                 from pydantic import BaseModel

#                 class AnswerWithJustification(BaseModel):
#                     '''An answer to the user question along with justification for the answer.'''
#                     answer: str
#                     justification: str

#                 llm = OllamaFunctions(model="phi3", format="json", temperature=0)
#                 structured_llm = llm.with_structured_output(AnswerWithJustification)

#                 structured_llm.invoke("What weighs more a pound of bricks or a pound of feathers")

#                 # -> AnswerWithJustification(
#                 #     answer='They weigh the same',
#                 #     justification='Both a pound of bricks and a pound of feathers weigh one pound. The weight is the same, but the volume or density of the objects may differ.'
#                 # )

#         Example: Pydantic schema (include_raw=True):
#             .. code-block:: python

#                 from langchain_experimental.llms import OllamaFunctions
#                 from pydantic import BaseModel

#                 class AnswerWithJustification(BaseModel):
#                     '''An answer to the user question along with justification for the answer.'''
#                     answer: str
#                     justification: str

#                 llm = OllamaFunctions(model="phi3", format="json", temperature=0)
#                 structured_llm = llm.with_structured_output(AnswerWithJustification, include_raw=True)

#                 structured_llm.invoke("What weighs more a pound of bricks or a pound of feathers")
#                 # -> {
#                 #     'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ao02pnFYXD6GN1yzc0uXPsvF', 'function': {'arguments': '{"answer":"They weigh the same.","justification":"Both a pound of bricks and a pound of feathers weigh one pound. The weight is the same, but the volume or density of the objects may differ."}', 'name': 'AnswerWithJustification'}, 'type': 'function'}]}),
#                 #     'parsed': AnswerWithJustification(answer='They weigh the same.', justification='Both a pound of bricks and a pound of feathers weigh one pound. The weight is the same, but the volume or density of the objects may differ.'),
#                 #     'parsing_error': None
#                 # }

#         Example: dict schema (method="include_raw=False):
#             .. code-block:: python

#                 from langchain_experimental.llms import OllamaFunctions, convert_to_ollama_tool
#                 from pydantic import BaseModel

#                 class AnswerWithJustification(BaseModel):
#                     '''An answer to the user question along with justification for the answer.'''
#                     answer: str
#                     justification: str

#                 dict_schema = convert_to_ollama_tool(AnswerWithJustification)
#                 llm = OllamaFunctions(model="phi3", format="json", temperature=0)
#                 structured_llm = llm.with_structured_output(dict_schema)

#                 structured_llm.invoke("What weighs more a pound of bricks or a pound of feathers")
#                 # -> {
#                 #     'answer': 'They weigh the same',
#                 #     'justification': 'Both a pound of bricks and a pound of feathers weigh one pound. The weight is the same, but the volume and density of the two substances differ.'
#                 # }


#         """  # noqa: E501
#         if kwargs:
#             raise ValueError(f"Received unsupported arguments {kwargs}")
#         is_pydantic_schema = _is_pydantic_class(schema)
#         if schema is None:
#             raise ValueError(
#                 "schema must be specified when method is 'function_calling'. "
#                 "Received None."
#             )
#         llm = self.bind_tools(tools=[schema], format="json")
#         if is_pydantic_schema:
#             output_parser: OutputParserLike = PydanticOutputParser(  # type: ignore[type-var]
#                 pydantic_object=schema  # type: ignore[arg-type]
#             )
#         else:
#             output_parser = JsonOutputParser()

#         parser_chain = RunnableLambda(parse_response) | output_parser
#         if include_raw:
#             parser_assign = RunnablePassthrough.assign(
#                 parsed=itemgetter("raw") | parser_chain, parsing_error=lambda _: None
#             )
#             parser_none = RunnablePassthrough.assign(parsed=lambda _: None)
#             parser_with_fallback = parser_assign.with_fallbacks(
#                 [parser_none], exception_key="parsing_error"
#             )
#             return RunnableMap(raw=llm) | parser_with_fallback
#         else:
#             return llm | parser_chain

#     def _generate(
#         self,
#         messages: List[BaseMessage],
#         stop: Optional[List[str]] = None,
#         run_manager: Optional[CallbackManagerForLLMRun] = None,
#         **kwargs: Any,
#     ) -> ChatResult:
#         functions = kwargs.get("functions", [])
#         if "functions" in kwargs:
#             del kwargs["functions"]
#         if "function_call" in kwargs:
#             functions = [
#                 fn for fn in functions if fn["name"] == kwargs["function_call"]["name"]
#             ]
#             if not functions:
#                 raise ValueError(
#                     "If `function_call` is specified, you must also pass a "
#                     "matching function in `functions`."
#                 )
#             del kwargs["function_call"]
#         functions = [convert_to_ollama_tool(fn) for fn in functions]
#         functions.append(DEFAULT_RESPONSE_FUNCTION)
#         system_message_prompt_template = SystemMessagePromptTemplate.from_template(
#             self.tool_system_prompt_template
#         )
#         system_message = system_message_prompt_template.format(
#             tools=json.dumps(functions, indent=2)
#         )
#         response_message = super()._generate(
#             [system_message] + messages, stop=stop, run_manager=run_manager, **kwargs
#         )
#         chat_generation_content = response_message.generations[0].text
#         if not isinstance(chat_generation_content, str):
#             raise ValueError("OllamaFunctions does not support non-string output.")
#         try:
#             parsed_chat_result = json.loads(chat_generation_content)
#         except json.JSONDecodeError:
#             raise ValueError(
#                 f"""'{self.model}' did not respond with valid JSON. 
#                 Please try again. 
#                 Response: {chat_generation_content}"""
#             )
#         called_tool_name = (
#             parsed_chat_result["tool"] if "tool" in parsed_chat_result else None
#         )
#         called_tool = next(
#             (fn for fn in functions if fn["name"] == called_tool_name), None
#         )
#         if (
#             called_tool is None
#             or called_tool["name"] == DEFAULT_RESPONSE_FUNCTION["name"]
#         ):
#             if (
#                 "tool_input" in parsed_chat_result
#                 and "response" in parsed_chat_result["tool_input"]
#             ):
#                 response = parsed_chat_result["tool_input"]["response"]
#             elif "response" in parsed_chat_result:
#                 response = parsed_chat_result["response"]
#             else:
#                 raise ValueError(
#                     f"Failed to parse a response from {self.model} output: "
#                     f"{chat_generation_content}"
#                 )
#             return ChatResult(
#                 generations=[
#                     ChatGeneration(
#                         message=AIMessage(
#                             content=response,
#                         )
#                     )
#                 ]
#             )

#         called_tool_arguments = (
#             parsed_chat_result["tool_input"]
#             if "tool_input" in parsed_chat_result
#             else {}
#         )

#         response_message_with_functions = AIMessage(
#             content="",
#             tool_calls=[
#                 ToolCall(
#                     name=called_tool_name,
#                     args=called_tool_arguments if called_tool_arguments else {},
#                     id=f"call_{str(uuid.uuid4()).replace('-', '')}",
#                 )
#             ],
#         )

#         return ChatResult(
#             generations=[ChatGeneration(message=response_message_with_functions)]
#         )

#     async def _agenerate(
#         self,
#         messages: List[BaseMessage],
#         stop: Optional[List[str]] = None,
#         run_manager: Optional[AsyncCallbackManagerForLLMRun] = None,
#         **kwargs: Any,
#     ) -> ChatResult:
#         functions = kwargs.get("functions", [])
#         if "functions" in kwargs:
#             del kwargs["functions"]
#         if "function_call" in kwargs:
#             functions = [
#                 fn for fn in functions if fn["name"] == kwargs["function_call"]["name"]
#             ]
#             if not functions:
#                 raise ValueError(
#                     "If `function_call` is specified, you must also pass a "
#                     "matching function in `functions`."
#                 )
#             del kwargs["function_call"]
#         elif not functions:
#             functions.append(DEFAULT_RESPONSE_FUNCTION)
#         if _is_pydantic_class(functions[0]):
#             functions = [convert_to_ollama_tool(fn) for fn in functions]
#         system_message_prompt_template = SystemMessagePromptTemplate.from_template(
#             self.tool_system_prompt_template
#         )
#         system_message = system_message_prompt_template.format(
#             tools=json.dumps(functions, indent=2)
#         )
#         response_message = await super()._agenerate(
#             [system_message] + messages, stop=stop, run_manager=run_manager, **kwargs
#         )
#         chat_generation_content = response_message.generations[0].text
#         if not isinstance(chat_generation_content, str):
#             raise ValueError("OllamaFunctions does not support non-string output.")
#         try:
#             parsed_chat_result = json.loads(chat_generation_content)
#         except json.JSONDecodeError:
#             raise ValueError(
#                 f"""'{self.model}' did not respond with valid JSON. 
#                 Please try again. 
#                 Response: {chat_generation_content}"""
#             )
#         called_tool_name = parsed_chat_result["tool"]
#         called_tool_arguments = parsed_chat_result["tool_input"]
#         called_tool = next(
#             (fn for fn in functions if fn["name"] == called_tool_name), None
#         )
#         if called_tool is None:
#             raise ValueError(
#                 f"Failed to parse a function call from {self.model} output: "
#                 f"{chat_generation_content}"
#             )
#         if called_tool["name"] == DEFAULT_RESPONSE_FUNCTION["name"]:
#             return ChatResult(
#                 generations=[
#                     ChatGeneration(
#                         message=AIMessage(
#                             content=called_tool_arguments["response"],
#                         )
#                     )
#                 ]
#             )

#         response_message_with_functions = AIMessage(
#             content="",
#             additional_kwargs={
#                 "function_call": {
#                     "name": called_tool_name,
#                     "arguments": json.dumps(called_tool_arguments)
#                     if called_tool_arguments
#                     else "",
#                 },
#             },
#         )
#         return ChatResult(
#             generations=[ChatGeneration(message=response_message_with_functions)]
#         )

#     @property
#     def _llm_type(self) -> str:
#         return "ollama_functions"

In [ ]:

# Data for the dataframe
data = {
    'Character': [
        'Luke Skywalker', 'Darth Vader (Anakin)', 'Princess Leia Organa', 'Han Solo', 'Yoda',
        'Obi-Wan Kenobi', 'Palpatine (Emperor)', 'Chewbacca', 'R2-D2', 'C-3PO',
        'Boba Fett', 'Jabba the Hutt', 'Lando Calrissian', 'Padmé Amidala', 'Qui-Gon Jinn',
        'Mace Windu', 'Ahsoka Tano', 'Kylo Ren (Ben Solo)', 'Rey Skywalker', 'Finn',
        'Poe Dameron', 'Count Dooku', 'Darth Maul', 'Jango Fett', 'General Grievous',
        'Rose Tico', 'Captain Phasma', 'BB-8', 'Admiral Ackbar', 'Wedge Antilles',
        'Mon Mothma', 'Ezra Bridger', 'Sabine Wren', 'Kanan Jarrus', 'Grand Admiral Thrawn',
        'Jyn Erso', 'Cassian Andor', 'Chirrut Îmwe', 'Saw Gerrera', 'Orson Krennic'
    ],
    'First Movie': [
        'Star Wars: A New Hope', 'Star Wars: A New Hope', 'Star Wars: A New Hope', 'Star Wars: A New Hope',
        'Star Wars: The Empire Strikes Back', 'Star Wars: A New Hope', 'Star Wars: The Empire Strikes Back', 
        'Star Wars: A New Hope', 'Star Wars: A New Hope', 'Star Wars: A New Hope',
        'Star Wars: The Empire Strikes Back', 'Star Wars: Return of the Jedi', 'Star Wars: The Empire Strikes Back',
        'Star Wars: The Phantom Menace', 'Star Wars: The Phantom Menace', 'Star Wars: The Phantom Menace',
        'Star Wars: The Clone Wars', 'Star Wars: The Force Awakens', 'Star Wars: The Force Awakens', 'Star Wars: The Force Awakens','Star Wars: The Force Awakens',
        'Star Wars: Attack of the Clones', 'Star Wars: The Phantom Menace', 'Star Wars: Attack of the Clones', 
        'Star Wars: Revenge of the Sith', 'Star Wars: The Last Jedi', 'Star Wars: The Force Awakens', 
        'Star Wars: The Force Awakens', 'Star Wars: Return of the Jedi', 'Star Wars: A New Hope', 
        'Star Wars: Return of the Jedi', 'Star Wars Rebels', 'Star Wars Rebels', 'Star Wars Rebels', 
        'Star Wars Rebels', 'Rogue One: A Star Wars Story', 'Rogue One: A Star Wars Story', 'Rogue One: A Star Wars Story',
        'Star Wars: The Clone Wars', 'Rogue One: A Star Wars Story'
    ],
    'Year': [
        1977, 1977, 1977, 1977, 1980, 1977, 1980, 1977, 1977, 1977, 
        1980, 1983, 1980, 1999, 1999, 1999, 2008, 2015, 2015, 2015,
        2002, 1999, 2002, 2005, 2017, 2015, 2015, 1983, 1977, 1983, 
        2014, 2014, 2014, 2016, 2016, 2016, 2012, 2016, 2012, 2016
    ],
    'Background Summary': [
        """Luke Skywalker is one of the most iconic characters in cinematic history, originating from the classic “Star Wars” franchise. His journey from a farm boy on the remote desert planet of Tatooine to the galaxy’s greatest Jedi Knight is a tale of hope, perseverance, self-discovery, and the eternal struggle between good and evil. As the son of Anakin Skywalker (Darth Vader) and Padmé Amidala, Luke was born into a lineage that was pivotal to the fate of the galaxy. Hidden from his father after the fall of the Jedi Order and the rise of the Galactic Empire, Luke was raised by his Uncle Owen and Aunt Beru on Tatooine. Though initially unaware of his true heritage, Luke was always drawn to something greater than the life of a moisture farmer.
Luke’s journey truly begins when he encounters the droids R2-D2 and C-3PO, who carry a message from Princess Leia Organa (his twin sister, though he does not know it yet). This message leads Luke to the reclusive Jedi Master Obi-Wan Kenobi, who reveals the basics of the Force to him and provides him with his father’s lightsaber. This meeting sets Luke on a path that will not only change his life but also alter the course of the galaxy. After his uncle and aunt are killed by Imperial stormtroopers, Luke has nothing left on Tatooine and decides to join Obi-Wan on a mission to help Princess Leia and the Rebel Alliance.

Throughout the original trilogy, Luke undergoes significant growth, evolving from an eager but inexperienced young man to a wise and powerful Jedi Knight. His journey is heavily influenced by his mentors, Obi-Wan Kenobi and Yoda. In “Star Wars: A New Hope,” Luke joins forces with Princess Leia, Han Solo, and others to destroy the Empire’s massive superweapon, the Death Star. His piloting skills in the battle showcase his natural talent and connection to the Force. It is during this time that he begins to embrace the idea of becoming a Jedi, a path that will test him physically, mentally, and spiritually.

In “The Empire Strikes Back,” Luke’s training under Yoda on Dagobah is one of the most iconic sequences in Star Wars. Here, Luke learns more about the Force, but he also confronts his own fears and doubts. The revelation that Darth Vader is his father is one of the most shocking moments in film history, and it profoundly impacts Luke. He grapples with the knowledge that the man responsible for so much pain and suffering in the galaxy is also the man who fathered him. This revelation makes Luke’s journey even more complicated, as he must decide whether to follow the path of the Jedi or risk falling to the dark side, as his father did.

In “Return of the Jedi,” Luke emerges as a fully-fledged Jedi Knight. His mission to redeem his father and defeat the Emperor becomes the central focus of his story. Luke’s compassion and refusal to give up on Anakin Skywalker ultimately lead to the redemption of his father and the downfall of the Sith. By rejecting the Emperor’s temptation to turn to the dark side, Luke proves himself to be the embodiment of the Jedi ideals of hope, selflessness, and the belief in the goodness of others. His final confrontation with Darth Vader is not just a physical battle but a test of his character and faith in the light side of the Force.

After the defeat of the Empire, Luke goes on to rebuild the Jedi Order, though this process is fraught with difficulties. His journey is expanded in the sequel trilogy, where we see an older, disillusioned Luke, who has gone into self-imposed exile after a disastrous attempt to train a new generation of Jedi. The rise of Kylo Ren (Ben Solo, Luke’s nephew) and the fall of his Jedi students deeply affects Luke, leading him to question the legacy of the Jedi and his role in the galaxy’s ongoing conflict. Despite his initial reluctance, Luke returns to the fight in “The Last Jedi” by projecting his Force presence to confront Kylo Ren and inspire the Resistance. His sacrifice allows the remnants of the Resistance to escape, and his legend grows even further, solidifying his place as one of the galaxy’s greatest heroes.

Luke Skywalker’s legacy continues to influence the Star Wars universe. He is the quintessential hero, a symbol of hope and perseverance in the face of overwhelming odds. From his humble beginnings on Tatooine to his role as the redeemer of Anakin Skywalker and savior of the galaxy, Luke’s character arc is one of the most well-developed and beloved in all of science fiction and fantasy.

This version of Luke Skywalker draws upon his appearances in the original trilogy (“A New Hope,” “The Empire Strikes Back,” and “Return of the Jedi”), as well as his expanded role in the sequel trilogy (“The Force Awakens,” “The Last Jedi,” and “The Rise of Skywalker”). His story is also explored in numerous novels, comic books, and animated series that flesh out his journey to restore the Jedi Order and maintain peace in the galaxy. Through these additional materials, fans get to see Luke in various stages of his life, from a young and idealistic Jedi-in-training to a wise but burdened mentor figure who ultimately finds redemption and peace in his final moments.

Luke Skywalker’s significance to the Star Wars mythos cannot be overstated. He represents the core themes of the saga: the battle between light and dark, the importance of family and legacy, and the enduring belief that even the darkest individuals can find redemption. His story is one of hope, not just for the galaxy, but for all those who seek to overcome their inner demons and rise above adversity.

This example provides a comprehensive character background with deep insights into Luke Skywalker’s journey and significance. Each character in your DataFrame will have a similarly detailed and expanded summary that goes beyond their initial description, pulling from movies, shows, and other Star Wars media.'
""",
        """Anakin Skywalker, who later becomes Darth Vader, is one of the most complex and tragic characters in the Star Wars saga. Born as a slave on Tatooine, Anakin’s potential in the Force was recognized at a young age by Jedi Master Qui-Gon Jinn, who believed Anakin to be the Chosen One, prophesized to bring balance to the Force. Qui-Gon’s belief in Anakin was so strong that, even as he lay dying, he implored his apprentice, Obi-Wan Kenobi, to train the boy as a Jedi. This began Anakin’s journey from an innocent, yet extraordinarily talented boy, to the dark enforcer of the Galactic Empire, Darth Vader.

Anakin’s early life was filled with hardship. Raised by his mother, Shmi Skywalker, in the harsh conditions of the desert planet, he yearned for freedom and adventure. His natural instincts and connection to the Force allowed him to become a skilled podracer, a rare feat for a human, and eventually, it was his talents that brought him into contact with Qui-Gon and Obi-Wan. However, Anakin’s fear of losing those he loved—stemming from his separation from his mother—was a driving force behind many of his decisions, which would eventually lead him to fall to the dark side.

As a Jedi, Anakin was exceptionally powerful, but his impulsive nature, emotional instability, and desire for control made him a difficult student for the Jedi Council. Although he earned a place as Obi-Wan’s apprentice, Anakin frequently questioned the Jedi’s teachings, particularly their emphasis on emotional detachment. This became especially evident when he secretly married Padmé Amidala, the former Queen of Naboo, an action that was strictly forbidden by the Jedi Code.

The most significant turning point in Anakin’s life came during the Clone Wars, a galactic conflict that spanned several years. During the war, Anakin earned a reputation as a fearless warrior and a brilliant strategist, leading countless missions for the Republic. However, his experiences in the war further fed his anger, impatience, and fear of loss. His worst fears were realized when he began having visions of Padmé dying in childbirth, similar to the dreams that foreshadowed his mother’s death. Desperate to prevent this, Anakin was seduced by Emperor Palpatine (then known as Chancellor Palpatine), who promised that the dark side of the Force could save Padmé’s life.

Palpatine, who had been manipulating Anakin for years, finally convinced him to turn against the Jedi. In his new identity as Darth Vader, Anakin slaughtered many of the Jedi, including younglings, in the infamous Jedi Temple massacre. His ultimate betrayal was in his duel with Obi-Wan Kenobi on the volcanic planet of Mustafar. The battle left Anakin horrifically injured and burned, and he was left for dead by Obi-Wan. Rescued by Palpatine, Anakin was placed in the black, mechanical armor that would become synonymous with Darth Vader, a walking symbol of the Emperor’s iron grip on the galaxy.

As Darth Vader, Anakin Skywalker became the Emperor’s right-hand enforcer, leading the Empire’s forces and hunting down the remaining Jedi. He was feared across the galaxy as the epitome of darkness, brutality, and power. The Emperor’s manipulation had turned Anakin into a tool of the dark side, and he lived in anguish, believing that his actions had caused Padmé’s death. This belief, along with his physical and emotional torment, made him one of the most feared figures in the galaxy. Yet, beneath the mask and armor, a small part of the compassionate and idealistic Anakin still remained, though buried deep within the monstrous visage of Darth Vader.

Vader’s redemption came through his son, Luke Skywalker, whose existence he was unaware of for many years. When Luke confronted Vader, he saw the good that still lingered within his father. In their final confrontation aboard the second Death Star, Luke’s refusal to turn to the dark side, even under the Emperor’s manipulation, inspired Anakin to finally break free of Palpatine’s control. In one final act of redemption, Anakin killed the Emperor, sacrificing his life in the process. His final moments were spent reconciled with his son, knowing that Luke had saved him from the darkness.

Anakin Skywalker’s story is a tale of immense tragedy and ultimate redemption. His fall to the dark side represents the dangers of unchecked emotions, fear, and the desire for control. Yet his eventual redemption through the love and persistence of his son serves as a powerful reminder of the enduring struggle between light and dark, and the possibility of salvation, even for those who seem beyond redemption.

Darth Vader remains one of the most iconic villains in film history, and his transformation from Anakin Skywalker to the Dark Lord of the Sith is central to the Star Wars mythos. His character’s journey—from a young boy with dreams of becoming a Jedi, to a feared enforcer of the Empire, and finally, to a redeemed father—illustrates the complexity of human nature and the eternal conflict between good and evil. Through his story, Star Wars explores themes of power, corruption, and the redemptive power of love and hope.""",
        """
Princess Leia Organa is one of the most iconic and influential characters in the Star Wars universe, symbolizing leadership, courage, and defiance in the face of tyranny. Born as Leia Amidala Skywalker, she is the twin sister of Luke Skywalker and the daughter of Anakin Skywalker (Darth Vader) and Padmé Amidala. To protect her from the Emperor and her father, she was adopted by Senator Bail Organa of Alderaan and raised as a member of the royal family. Growing up under the tutelage of her adoptive parents, Leia developed a strong sense of duty, justice, and compassion, eventually becoming a key figure in the Rebel Alliance and later the Resistance.

Leia’s story begins in “Star Wars: A New Hope,” where she is introduced as a young leader of the Rebellion, tasked with delivering the plans to the Death Star to the Rebel forces. Despite being captured by Darth Vader and subjected to interrogation by the Empire, Leia remains resilient, refusing to give up the location of the Rebel base. Her fortitude is evident when she witnesses the destruction of her home planet, Alderaan, at the hands of the Death Star, yet still does not falter in her mission.

Throughout the original trilogy, Leia’s role is multifaceted. She is not only a diplomatic leader and strategist for the Rebellion but also an active participant in missions, showcasing her bravery and combat skills. Her romance with Han Solo adds another dimension to her character, revealing her vulnerability and capacity for deep emotional connections. Despite her personal losses, including the revelation of her true parentage and the knowledge that Darth Vader is her father, Leia never wavers in her commitment to the cause of freedom.

In “The Empire Strikes Back” and “Return of the Jedi,” Leia’s leadership becomes even more critical. She takes part in the mission to rescue Han Solo from Jabba the Hutt, disguised as the bounty hunter Boushh. Later, she plays a vital role in the final battle against the Empire on the forest moon of Endor. It is during this time that she learns from Luke that they are siblings and that Vader is their father. Despite the shocking revelation, Leia remains steadfast, focused on the mission at hand, though the knowledge undoubtedly affects her.

Leia’s story continues in the sequel trilogy, where she transitions from a leader of the Rebel Alliance to the General of the Resistance. In “The Force Awakens,” it is revealed that her son, Ben Solo (Kylo Ren), has fallen to the dark side, leading to immense personal pain and tragedy for Leia. Despite this, she remains the backbone of the Resistance, guiding its members with wisdom and determination. Her relationship with Han Solo is strained by their shared grief over Ben’s fall, but their love and respect for one another endure.

Leia’s ultimate legacy is one of perseverance, hope, and resilience. Even in the face of overwhelming odds, personal loss, and betrayal, she remains committed to the ideals of the Rebellion and the Resistance. Her role as both a warrior and a diplomat highlights her versatility and strength. Leia’s leadership is not just about military strategy but also about inspiring others to believe in a better future, even when that future seems uncertain.

In “The Last Jedi” and “The Rise of Skywalker,” Leia’s influence extends beyond her direct actions. Though her health begins to fail, her presence as a guiding force remains strong. In her final moments, she reaches out to her son, Ben, through the Force, ultimately contributing to his redemption. Leia’s sacrifice symbolizes the enduring power of love and hope, themes that are central to the Star Wars saga.

Princess Leia’s character is a beacon of strength, not just in the Star Wars universe but also in popular culture. She is a feminist icon, representing a strong, independent woman who can hold her own in both political and combat arenas. Her legacy as a leader, a mother, and a symbol of resistance against tyranny endures through the generations of fans who continue to be inspired by her courage and determination.

Leia Organa’s story arc, from the early days of the Rebellion to her leadership in the Resistance, showcases her resilience and commitment to justice. She is a character who, despite immense personal and political challenges, never loses sight of her goals. Her ability to inspire those around her, her intelligence, and her deep compassion make her one of the most beloved characters in Star Wars. Leia’s journey reflects the broader themes of the Star Wars
        """,
        'Smuggler-turned-Rebel hero, pilot of the Millennium Falcon, and ally to Luke and Leia.',
        'Ancient Jedi Master who trained countless Jedi, including Luke Skywalker.',
        'Former Jedi Knight who trained Anakin and later guided Luke in the ways of the Force.',
        'Sith Lord and Emperor of the Galactic Empire, master of Darth Vader.',
        'Wookiee warrior and co-pilot of the Millennium Falcon, fiercely loyal to Han Solo.',
        'Astromech droid who served Anakin and later Luke, played key roles in many battles.',
        'Protocol droid fluent in many languages, companion to R2-D2, built by Anakin Skywalker.',
        'Notorious bounty hunter, son/clone of Jango Fett, and nemesis to Han Solo.',
        'Crime lord based on Tatooine, known for his dealings with bounty hunters and smugglers.',
        'Gambler, smuggler, and old friend of Han Solo who became a key figure in the Rebellion.',
        'Queen of Naboo and later Senator, mother to Luke and Leia, and wife of Anakin Skywalker.',
        'Jedi Master who discovered Anakin Skywalker and believed in the prophecy of the Chosen One.',
        'High-ranking Jedi Master on the Jedi Council, known for his skill with a lightsaber.',
        'Anakin Skywalker\'s Padawan, later a key figure in the Rebellion, and a skilled Jedi.',
        'Son of Han and Leia, conflicted Force user who turned to the dark side, became a Sith.',
        'Scavenger-turned-Jedi, granddaughter of Palpatine, who ultimately restores balance.',
        'Former Stormtrooper who defected to the Resistance and fought alongside Rey and Poe.',
        'Ace pilot of the Resistance, loyal to General Leia Organa, and a key leader in the fight.',
        'Former Jedi who became a Sith Lord, leading the Separatist movement in the Clone Wars.',
        'Sith apprentice to Darth Sidious, known for his double-bladed lightsaber and vengeance.',
        'Bounty hunter and template for the clone army, father/clone of Boba Fett.',
        'Cyborg commander of the Separatist droid army, feared for his lightsaber collection.',
        'Mechanic in the Resistance, inspired by her sister\'s sacrifice, and friend to Finn.',
        'Ruthless commander of the First Order\'s stormtroopers, known for her chrome armor.',
        'Spherical droid belonging to Poe Dameron, key in the Resistance\'s mission against the First Order.',
        'Mon Calamari leader in the Rebel Alliance, known for his tactical expertise ("It\'s a trap!").',
        'Rebel Alliance pilot who fought in major battles including the Death Star and Endor.',
        'Political leader of the Rebel Alliance, instrumental in forming the Rebellion against the Empire.',
        'Orphan-turned-Jedi who fought against the Empire and helped found the Rebel Alliance.',
        'Mandalorian warrior and artist, key member of the Ghost crew fighting the Empire.',
        'Former Jedi Knight who survived Order 66, mentor to Ezra Bridger, and Rebel leader.',
        'Brilliant Imperial strategist, known for his cunning and leadership in the Empire.',
        'Rebel fighter who led the mission to steal the Death Star plans, sacrificing herself in the process.',
        'Rebel spy and captain, key figure in the mission to acquire the Death Star plans.',
        'Blind warrior-monk who believes in the Force, ally to Jyn Erso and Cassian Andor.',
        'Rebel extremist who fought against both the Separatists and the Empire.',
        'Director of the Death Star project for the Empire, rival to Tarkin and other Imperial officers.'
    ],
    'Source':["source"]*40
}

# Create the dataframe
df = pd.DataFrame(data)

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 10,
    # length_function = len,
    # is_separator_regex = False,
)

In [40]:
text = """The text explains Anakin's and Leia's lives. Anakin’s early life was filled with hardship. 
Raised by his mother, Shmi Skywalker, in the harsh conditions of the desert planet, he yearned for freedom and adventure. 
His natural instincts and connection to the Force allowed him to become a skilled podracer, a rare feat for a human, and eventually, it was his talents that brought him into contact with Qui-Gon and Obi-Wan. 
However, Anakin’s fear of losing those he loved—stemming from his separation from his mother—was a driving force behind many of his decisions, which would eventually lead him to fall to the dark side. 
As a Jedi, Anakin was exceptionally powerful, but his impulsive nature, emotional instability, and desire for control made him a difficult student for the Jedi Council. 
Although he earned a place as Obi-Wan’s apprentice, Anakin frequently questioned the Jedi’s teachings, particularly their emphasis on emotional detachment. 
This became especially evident when he secretly married Padmé Amidala, the former Queen of Naboo, an action that was strictly forbidden by the Jedi Code. 
The most significant turning point in Anakin’s life came during the Clone Wars, a galactic conflict that spanned several years. 
During the war, Anakin earned a reputation as a fearless warrior and a brilliant strategist, leading countless missions for the Republic. 
However, his experiences in the war further fed his anger, impatience, and fear of loss. His worst fears were realized when he began having visions of Padmé dying in childbirth, similar to the dreams that foreshadowed his mother’s death. 
Desperate to prevent this, Anakin was seduced by Emperor Palpatine (then known as Chancellor Palpatine), who promised that the dark side of the Force could save Padmé’s life. Palpatine, who had been manipulating Anakin for years, finally convinced him to turn against the Jedi. 
In his new identity as Darth Vader, Anakin slaughtered many of the Jedi, including younglings, in the infamous Jedi Temple massacre. His ultimate betrayal was in his duel with Obi-Wan Kenobi on the volcanic planet of Mustafar. 
Anakin Skywalker’s story is a tale of immense tragedy and ultimate redemption. His fall to the dark side represents the dangers of unchecked emotions, fear, and the desire for control. 
Yet his eventual redemption through the love and persistence of his son serves as a powerful reminder of the enduring struggle between light and dark, and the possibility of salvation, even for those who seem beyond redemption. 
Darth Vader remains one of the most iconic villains in film history, and his transformation from Anakin Skywalker to the Dark Lord of the Sith is central to the Star Wars mythos. 
His character’s journey—from a young boy with dreams of becoming a Jedi, to a feared enforcer of the Empire, and finally, to a redeemed father—illustrates the complexity of human nature and the eternal conflict between good and evil. 
Through his story, Star Wars explores themes of power, corruption, and the redemptive power of love and hope."""
documents = [Document(page_content=text)]

In [44]:
llm = ChatOllama(model="llama3.1")
llm_transformer = LLMGraphTransformer(llm=llm)

In [45]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [46]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Darth Vader', type='Person', properties={}), Node(id='Padmé Amidala', type='Person', properties={}), Node(id='Qui-Gon and Obi-Wan', type='Group of People', properties={}), Node(id='Dark Side', type='Force', properties={}), Node(id='Countless missions', type='Achievement', properties={}), Node(id='Shmi Skywalker', type='Person', properties={}), Node(id='Anakin Skywalker', type='Person', properties={}), Node(id='Obi-Wan Kenobi', type='Person', properties={}), Node(id='Jedi', type='Order', properties={}), Node(id='Queen of Naboo', type='Title', properties={}), Node(id='The Force', type='Force', properties={}), Node(id='Republic', type='Government', properties={}), Node(id='Jedi Order', type='Order', properties={}), Node(id='Younglings', type='Group of People', properties={})]
Relationships:[Relationship(source=Node(id='Anakin Skywalker', type='Person', properties={}), target=Node(id='Shmi Skywalker', type='Person', properties={}), type='RAISED_BY', properties={}), Relation

In [43]:
cypher = """
MATCH (n)
DETACH DELETE n
"""
kg.query(cypher)

[]

In [48]:
kg.add_graph_documents(graph_documents)